# Set Your SQL Data Free with Kafka

Check that the necessary object have been set up

In [ ]:
USE GamePlayDB;

In [ ]:
SELECT DB_NAME();

In [ ]:
SELECT * FROM dbo.tb_GamePlay;

In [ ]:
SELECT * FROM dbo.tb_KafkaCluster;

In [ ]:
SELECT * FROM dbo.tb_KafkaEventSubscriber;

## Stored Procedures

Main procedure to log a wager

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_LogWager @UserID INT,
                                          @GameID INT,
                                          @WagerAmount bigint,
                                          @PayoutAmount bigint,
                                          @EventTime datetime2
AS
BEGIN
  IF(@EventTime IS NULL)
  BEGIN
    SET @EventTime = SYSUTCDATETIME();
  END 

  BEGIN TRY
    BEGIN TRAN
      INSERT INTO dbo.tb_GamePlay(UserID, GameID, WagerAmount, PayoutAmount, EventTime)
      VALUES(@UserID, @GameID, @WagerAmount, @PayoutAmount, @EventTime);
      --do some more tx "stuff" here
    COMMIT TRAN
                              
  END TRY
  BEGIN CATCH
    ROLLBACK TRAN;
  END CATCH

END  
GO

Procedure to emulate game-play

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_EmulateGamePlay @Loops int = 10,
                                                 @MinDelay int = 50,
                                                 @MaxDelay int = 500
AS
BEGIN                                        
  DECLARE @delay int;
  DECLARE @waitfor varchar(25);
  DECLARE @winMultiplier int;
  DECLARE @wager bigint;
  DECLARE @userId int;
  DECLARE @gameId int;
  DECLARE @payout bigint = 0;
  DECLARE @minUserId int = 100;
  DECLARE @maxUserId int = 125;
  DECLARE @minwager int = 250;
  DECLARE @maxwager int = 500000;
  DECLARE @minWinMult int = 1;
  DECLARE @maxWinMult int = 10;
  DECLARE @minGameId int = 1000;
  DECLARE @maxGameId int = 1050;
  DECLARE @minWinIndicator int = 1;
  DECLARE @maxWinIndicator int = 12;
  DECLARE @noLoops int = 0;
  DECLARE @eventTime datetime2;
  
  WHILE(@noLoops < @Loops)
  BEGIN

    SET @eventTime = SYSUTCDATETIME();
  -- get random values for delay between games, wager size, user id, game id,
  -- win multiplier
    SELECT @delay = @MinDelay + ROUND(RAND() * (@MaxDelay + 1 - @MinDelay), 0),
            @wager = (@minwager + ROUND(RAND() * (@maxwager + 1 - @minwager), 0)) / 100,
            @userId = @minUserId + ROUND(RAND() * (@maxUserId + 1 - @minUserId), 0),
            @gameId = @minGameId + ROUND(RAND() * (@maxGameId + 1 - @minGameId), 0),
            @winMultiplier = @minWinMult + ROUND(RAND() * (@maxWinMult + 1 - @minWinMult), 0)
    -- set up the waitfor variable
    SELECT @waitfor = FORMATMESSAGE('00:00:00.%i', @delay);
    --check if win
    IF(CAST((@minWinIndicator + ROUND(RAND() * (@maxWinIndicator + 1 - @minWinIndicator), 0)) AS int) % 3) = 0
    BEGIN
        SET @payout =   @wager * @winMultiplier;
    END

    EXEC dbo.pr_LogWager    @UserID = @userId,
                            @GameID = @gameId,
                            @WagerAmount = @wager,
                            @PayoutAmount = @payout,
                            @EventTime = @eventTime;
  
    SELECT @noLoops += 1, @delay = null, @wager = null, @userId = null, @gameId = null, @winMultiplier = null,  
         @waitfor = '', @payout = 0;

    WAITFOR DELAY @waitfor;

  END
END  

## Test it Works

In [ ]:
SET NOCOUNT ON;
EXECUTE dbo.pr_EmulateGamePlay @Loops = 3

In [ ]:
SELECT * FROM dbo.tb_GamePlay

In [ ]:
TRUNCATE TABLE dbo.tb_GamePlay